# Teacher's Assignment No. 15 - Q1

***Author:*** *Ofir Paz* $\qquad$ ***Version:*** *15.05.2024* $\qquad$ ***Course:*** *22961 - Deep Learning*

Welcome to the first question of the fifth assignment of the course *Deep Learning*. \
In this question we will implement the functionality of `nn.Conv2d`.

## Imports

First, we will import the required packages for this assignment
- [pytorch](https://pytorch.org/) - One of the most fundemental and famous tensor handling library.

In [1]:
from typing import Tuple
import torch  # pytorch.
import torch.nn as nn  # neural network module.
import torch.optim as optim  # optimization module.
import torch.nn.functional as F  # functional module.
import numpy as np  # numpy.
import torch.utils.data  # data handling module.
import matplotlib.pyplot as plt  # plotting module.

## Conv2d Implementation

In [2]:
class Conv2d(nn.Module):
    '''Conv2d implementation.

    Implements a 2D convolutional layer, without using any of the 
    functions from the module `torch.nn`.
    '''
    def __init__(self, in_channels: int = 1, out_channels: int = 1, 
                 kernel_size: Tuple[int, int] = (1, 1), stride: int = 1, padding: int = 0) -> None:
        '''Conv2d constructore

        Args:
            in_channels (int): number of input channels.
            out_channels (int): number of output channels.
            kernel_size (Tuple[int, int]): kernel size.
            stride (int): stride.
            padding (int): padding.

        Returns:
            None
        '''
        super(Conv2d, self).__init__()

        # initialize kernels and bias.
        self.kernels: torch.Tensor = nn.Parameter(torch.rand(out_channels, in_channels, *kernel_size))
        self.bias: torch.Tensor = nn.Parameter(torch.zeros(out_channels))

        # save parameters.
        self.p, self.q = kernel_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.padding = padding
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        '''
        Forward pass.

        Args:
            x (torch.Tensor): input tensor. Assumes shape (N, C, H, W).

        Returns:
            torch.Tensor: output tensor, with shape (N, out_channels, H', W').
        '''
        assert x.dim() == 4, \
            f'Input tensor must have 4 dimensions. Got {x.dim()} dimensions.'
        assert x.size(1) == self.in_channels, \
            f'Input tensor must have the same number of channels as the layer. Got {x.size(1)} channels.'
        assert x.size(2) >= self.p, \
            f'Input tensor must have a height greater than the kernel size. Got {x.size(2)} height.'
        assert x.size(3) >= self.q, \
            f'Input tensor must have a width greater than the kernel size. Got {x.size(3)} width.'

        out_shape = lambda n, f: (n + 2 * self.padding - f) // self.stride + 1
        output = torch.zeros(x.size(0), self.out_channels, 
                             out_shape(x.size(2), self.p), out_shape(x.size(3), self.q))
        
        if self.padding > 0:
            x_padded = torch.zeros(x.size(0), x.size(1), 
                                   x.size(2) + 2 * self.padding, x.size(3) + 2 * self.padding)
            x_padded[:, :, self.padding : -self.padding, self.padding : -self.padding] = x
            x = x_padded
        
        for c in range(output.size(1)):
            for h in range(output.size(2)):
                for w in range(output.size(3)):
                    sh, sw = h * self.stride, w * self.stride
                    sub_img = x[:, :, sh : sh + self.p, sw : sw + self.q]
                    output[:, c, h, w] = (sub_img * self.kernels[c]).sum(dim=(1,2,3)) + self.bias[c]

        return output

Validating the implementation

In [3]:
# Unit tests for the Conv2d class.
class Conv2dTestCase():
    def __init__(self):
        # Set random seed for reproducibility.
        torch.manual_seed(0)
        np.random.seed(0)
    
        # List of dicts containing parameters for input tensors.
        self.tensor_inputs = [
            {'size': (1, 6, 32, 32)},
            {'size': (16, 8, 42, 40)},
            {'size': (10, 2, 53, 34)},
            {'size': (1, 10, 20, 20)},
            {'size': (2, 4, 10, 10)}
        ]

        # List of dictionaries containing the parameters for the Conv2d layer.
        self.conv_inputs = [
            {'in_channels': 6, 'out_channels': 10, 'kernel_size': (3, 3), 'stride': 1, 'padding': 1},
            {'in_channels': 8, 'out_channels': 2, 'kernel_size': (5, 4), 'stride': 2, 'padding': 1},
            {'in_channels': 2, 'out_channels': 2, 'kernel_size': (1, 1), 'stride': 1, 'padding': 2},
            {'in_channels': 10, 'out_channels': 10, 'kernel_size': (6, 8), 'stride': 3, 'padding': 3},
            {'in_channels': 4, 'out_channels': 5, 'kernel_size': (5, 5), 'stride': 2, 'padding': 4}
        ]

    def test_forward(self) -> None:
        '''Test the forward method of the Conv2d class.'''

        for i, (conv_input, tensor_input) in enumerate(zip(self.conv_inputs, self.tensor_inputs)):
            # Create Conv2d instance.
            conv2d = Conv2d(**conv_input)

            # Create nn.Conv2d instance.
            nn_conv2d = nn.Conv2d(**conv_input)

            # Set both parameters to be the same.
            nn_conv2d.weight.data = conv2d.kernels
            nn_conv2d.bias.data = conv2d.bias  # type: ignore

            # Create input tensor.
            input_tensor = torch.rand(**tensor_input)

            # Forward pass through Conv2d.
            output = conv2d(input_tensor)

            # Forward pass through nn.Conv2d.
            nn_output = nn_conv2d(input_tensor)

            # Compare the outputs.
            assert torch.allclose(output, nn_output) == True, \
                f"""Conv2d forward method *failed* with input shape {input_tensor.size()} 
                and parameters \n    {conv_input}."""
            
            print(f'[{i+1}] Conv2d forward method *passed* with input shape {input_tensor.size()} '
                  f'and parameters \n    {conv_input}.')

tester = Conv2dTestCase()
tester.test_forward()

[1] Conv2d forward method *passed* with input shape torch.Size([1, 6, 32, 32]) and parameters 
    {'in_channels': 6, 'out_channels': 10, 'kernel_size': (3, 3), 'stride': 1, 'padding': 1}.
[2] Conv2d forward method *passed* with input shape torch.Size([16, 8, 42, 40]) and parameters 
    {'in_channels': 8, 'out_channels': 2, 'kernel_size': (5, 4), 'stride': 2, 'padding': 1}.
[3] Conv2d forward method *passed* with input shape torch.Size([10, 2, 53, 34]) and parameters 
    {'in_channels': 2, 'out_channels': 2, 'kernel_size': (1, 1), 'stride': 1, 'padding': 2}.
[4] Conv2d forward method *passed* with input shape torch.Size([1, 10, 20, 20]) and parameters 
    {'in_channels': 10, 'out_channels': 10, 'kernel_size': (6, 8), 'stride': 3, 'padding': 3}.
[5] Conv2d forward method *passed* with input shape torch.Size([2, 4, 10, 10]) and parameters 
    {'in_channels': 4, 'out_channels': 5, 'kernel_size': (5, 5), 'stride': 2, 'padding': 4}.
